In [1]:
from train_inception import feature_extraction_InV3, train_last_layer

In [12]:
from keras.preprocessing import image
from keras.layers import Dense,GlobalAveragePooling2D,BatchNormalization
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras import backend as K
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping

In [36]:
from keras.applications.inception_v3 import InceptionV3

In [3]:
img_width=299
img_height = 299
train_data_dir = "data/train"
num_image=1439
epochs = 10

In [5]:
train_X, train_y=feature_extraction_InV3(img_width, img_height,
                        train_data_dir,
                        num_image,
                        epochs)

Found 1439 images belonging to 4 classes.
96/96 [==============================] - 660s 7s/step
(1439, 2048) (1439, 4)


In [91]:
sgd = optimizers.SGD(lr=0.01)

In [92]:
my_model = Sequential()
my_model.add(BatchNormalization(input_shape=train_X.shape[1:]))
my_model.add(Dense(256, activation = "relu"))
my_model.add(Dense(4, activation='softmax'))

my_model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])
#early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
my_model.fit(train_X, train_y,epochs=100,batch_size=30,validation_split=0.1,verbose=1)

Train on 1295 samples, validate on 144 samples
Epoch 1/100
1295/1295 [==============================] - 7s 5ms/step - loss: 0.8915 - acc: 0.6409 - val_loss: 1.5084 - val_acc: 0.2917
Epoch 2/100
1295/1295 [==============================] - 1s 432us/step - loss: 0.4599 - acc: 0.8440 - val_loss: 1.4535 - val_acc: 0.3542
Epoch 3/100
1295/1295 [==============================] - 1s 480us/step - loss: 0.3447 - acc: 0.8842 - val_loss: 1.2750 - val_acc: 0.4444
Epoch 4/100
1295/1295 [==============================] - 1s 428us/step - loss: 0.2741 - acc: 0.9189 - val_loss: 1.0074 - val_acc: 0.5347
Epoch 5/100
1295/1295 [==============================] - 1s 470us/step - loss: 0.2148 - acc: 0.9382 - val_loss: 0.9026 - val_acc: 0.5764
Epoch 6/100
1295/1295 [==============================] - 1s 517us/step - loss: 0.1978 - acc: 0.9444 - val_loss: 1.0213 - val_acc: 0.5486
Epoch 7/100
1295/1295 [==============================] - 1s 437us/step - loss: 0.1610 - acc: 0.9645 - val_loss: 0.8386 - val_acc: 0.6

Epoch 60/100
1295/1295 [==============================] - 1s 417us/step - loss: 0.0110 - acc: 0.9992 - val_loss: 1.3584 - val_acc: 0.5694
Epoch 61/100
1295/1295 [==============================] - 1s 417us/step - loss: 0.0105 - acc: 1.0000 - val_loss: 1.2391 - val_acc: 0.6111
Epoch 62/100
1295/1295 [==============================] - 1s 442us/step - loss: 0.0101 - acc: 1.0000 - val_loss: 1.3512 - val_acc: 0.5764
Epoch 63/100
1295/1295 [==============================] - 1s 417us/step - loss: 0.0105 - acc: 0.9992 - val_loss: 1.4135 - val_acc: 0.5486
Epoch 64/100
1295/1295 [==============================] - 1s 417us/step - loss: 0.0116 - acc: 0.9992 - val_loss: 1.4330 - val_acc: 0.5625
Epoch 65/100
1295/1295 [==============================] - 1s 418us/step - loss: 0.0106 - acc: 0.9992 - val_loss: 1.3706 - val_acc: 0.5625
Epoch 66/100
1295/1295 [==============================] - 1s 440us/step - loss: 0.0101 - acc: 1.0000 - val_loss: 1.5211 - val_acc: 0.5347
Epoch 67/100
1295/1295 [==========

In [96]:
test_generator=ImageDataGenerator(rescale=1. / 255).flow_from_directory(test_data_dir,
target_size = (299, 299),
batch_size = 15,
class_mode = "categorical",
shuffle=False)
y_test=test_generator.classes
test_generator.reset
X_test=model.predict_generator(test_generator,verbose=1)
y_pred=my_model.predict(X_test,verbose=1)
accuracy=accuracy_score(y_test,y_pred.argmax(axis=1))
log_loss=log_loss(y_test,y_pred)
print ("Accuracy score: {}".format (accuracy))
print ("Log_loss score: {}".format (log_loss))

Found 352 images belonging to 4 classes.
352/352 [==============================] - 0s 282us/step
Accuracy score: 0.8522727272727273
Log_loss score: 0.5200667234862363


In [ ]:
my_model.save("inV3_last_layer.h5")

In [30]:
test_data_dir="data/test"

In [31]:
test_generator=ImageDataGenerator(rescale=1. / 255).flow_from_directory(test_data_dir,
target_size = (299, 299),
batch_size = 15, 
class_mode = "categorical",
                                                     shuffle=False)

Found 352 images belonging to 4 classes.


In [43]:
y_test=test_generator.classes

In [50]:
test_generator.reset

<bound method Iterator.reset of <keras.preprocessing.image.DirectoryIterator object at 0x1273da438>>

In [37]:
base_model = InceptionV3(input_shape=(299, 299, 3),
                              weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)

model = Model(inputs=base_model.input, outputs=x)

In [38]:
X_test=model.predict_generator(test_generator,verbose=1)

24/24 [==============================] - 176s 7s/step


In [93]:
y_pred=my_model.predict(X_test,verbose=1)

352/352 [==============================] - 3s 8ms/step


In [89]:
y_pred

array([[9.9999452e-01, 1.1952590e-06, 3.4797871e-08, 4.2500669e-06],
       [9.9999344e-01, 7.7219745e-08, 3.2914915e-07, 6.0775378e-06],
       [4.9965445e-02, 2.2622485e-01, 5.1610544e-03, 7.1864861e-01],
       ...,
       [5.4245493e-03, 1.1264270e-03, 9.2897457e-01, 6.4474508e-02],
       [4.8294063e-03, 2.3817454e-01, 7.5519055e-01, 1.8055416e-03],
       [1.9877350e-06, 9.7551733e-01, 2.4129853e-02, 3.5086213e-04]],
      dtype=float32)

In [87]:
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [84]:
from sklearn.metrics import accuracy_score,log_loss

In [94]:
accuracy_score(y_test,y_pred.argmax(axis=1))

0.8522727272727273

In [95]:
log_loss(y_test,y_pred)

0.5200667234862363

In [ ]:

my_model = Sequential()
my_model.add(BatchNormalization(input_shape=train_X.shape[1:]))
my_model.add(Dense(256, activation = "relu"))
my_model.add(Dense(4, activation='softmax'))

my_model.compile(optimizer=sgd, loss='categorical_crossentropy',metrics=['accuracy'])
#early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
my_model.fit(train_X, train_y,epochs=200,batch_size=15,validation_split=0.1,verbose=1)